In [1]:
import os
os.chdir("..")
from src.SparkEnv import sparkSess
import pyspark.sql.functions as F
from pyspark.sql import DataFrameStatFunctions as statFunc
import pandas as pd

Successfully imported Spark Context and Spark Configuration


In [24]:
!sudo python3 -m pip install numpy==1.15.4

    100% |████████████████████████████████| 13.9MB 97kB/s  eta 0:00:01
  Found existing installation: numpy 1.19.1
    Uninstalling numpy-1.19.1:
      Successfully uninstalled numpy-1.19.1
You are using pip version 9.0.3, however version 20.2.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, FloatType
from pyspark.sql.functions import UserDefinedFunction, udf, struct
from pyspark.sql.functions import col, when

import pyspark.sql.functions as F
import numpy as np

#import matplotlib.pyplot as plt
#import seaborn as sns
#
#sns.set()

In [3]:
from pyspark.sql import Row
from pyspark.sql.types import StringType, DataType
from pyspark.sql.functions import col, when, lit, concat_ws
from pyspark.sql import SparkSession

In [4]:
pre = "hdfs:///input"

In [5]:
!hadoop fs -ls $pre/

Found 12 items
-rw-r--r--   1 hadoop hadoop    6260261 2020-08-12 16:07 hdfs:///input/featureIndex.csv
drwxr-xr-x   - hadoop hadoop          0 2020-08-12 16:06 hdfs:///input/feature_data
-rw-r--r--   1 hadoop hadoop 7653991756 2020-08-12 16:09 hdfs:///input/ff_dsp_non_addressable_model_acx_tbl.csv
-rw-r--r--   1 hadoop hadoop 2530566106 2020-08-12 16:19 hdfs:///input/ff_dsp_non_addressable_model_aff_tbl.csv
drwxr-xr-x   - hadoop hadoop          0 2020-08-12 16:08 hdfs:///input/gdsa_conversion_data
drwxr-xr-x   - hadoop hadoop          0 2020-08-12 16:08 hdfs:///input/gdsa_data
drwxr-xr-x   - hadoop hadoop          0 2020-08-12 16:10 hdfs:///input/gdsa_data_acxiom_enh
drwxr-xr-x   - hadoop hadoop          0 2020-08-12 16:11 hdfs:///input/gdsa_data_piq
drwxr-xr-x   - hadoop hadoop          0 2020-08-12 16:10 hdfs:///input/gdsa_data_tu
drwxr-xr-x   - hadoop hadoop          0 2020-08-12 16:11 hdfs:///input/gdsa_data_zcodes
-rw-r--r--   1 hadoop hadoop 1054158304 2020-08-12 16:07 hdfs:///in

In [7]:
conversions = sparkSess.read.csv('hdfs:///input/gdsa_conversion_data', header= False).withColumn('Convertion', F.lit(1))  \
                                         .withColumnRenamed('_c2', 'EMAIL_ADDRESS_MD5')

+----------------+----------------+--------------------+----------+
|             _c0|             _c1|   EMAIL_ADDRESS_MD5|Convertion|
+----------------+----------------+--------------------+----------+
|07DPUS034TKKDPL3|07DPUS02VVVN4FTW|1866864cc75b4c559...|         1|
|07DPUS03ZVDM08L2|07DPUS1155V4LHLY|1868a51c468a6706b...|         1|
|07DPUS13HRSDKX79|07DPUS02ZCMMXJBE|1868d069aba4d307d...|         1|
|07DPUS03VP4VD368|07DPUS1179GDVR7Q|1869eda66f9acaae9...|         1|
|07DPUS03MPH4J35P|07DPUS01MZVR3EEH|186b2441c7d2183f2...|         1|
+----------------+----------------+--------------------+----------+
only showing top 5 rows



In [13]:
#conversions.count()

8632594

In [13]:
F.when

<function pyspark.sql.functions.when(condition, value)>

In [17]:
#!hadoop fs -put ./notebooks/tu_mapping.csv hdfs:///input/     #to put csv in hdfs

In [8]:
pre = "hdfs:///input"
acxiom_path= f"{pre}/ff_dsp_non_addressable_model_acx_tbl.csv"
tu_path= 'hdfs:///input/gdsa_data_tu/'
all_rfi_path = f"{pre}/user_requests_10mil"
md5_rfi_mappings_path= f"{pre}/md5_match_data.csv/"

tu_mapping_path= 'hdfs:///input/tu_mapping.csv'
conversion_path= 'hdfs:///input/gdsa_conversion_data'

## only needed to export
output_path= 'hdfs:///output_md5_mapping/'

In [21]:
# temp = mapping.get_final_mapping()
# temp.persist()
# temp.agg(F.countDistinct("RFI"), F.countDistinct("MD5")).show()

### Integrating the class with Convertion mapping

In [6]:
class Mapping_rfi_md5_conv:
    def __init__(self, 
                 acxiom_path, 
                 tu_path,
                 all_rfi_path,
                 md5_rfi_mappings_path,
                 conversion_path,
                 output_path):  
        
        self.na_values = ['-99', '\\N', '\\\\N', 'Missing', 'missing', 'NA', '?', '.', 'NULL', '', ' ']
        
        self.acxiom = sparkSess.read.csv(acxiom_path, header=True, nullValue= self.na_values)
        self.tu = sparkSess.read.csv(tu_path, header=True, nullValue= self.na_values)
        self.joined_records_DF = self.load_rfi_md5_mapping(all_rfi_path, md5_rfi_mappings_path)
        self.conversion_path= conversion_path
        self.output_path = output_path
  
    def execute(self):
        md5_with_count_missing_DF = self.generate_md5_count_missing()
        ranked_DF = self.filter_by_count_missing_rank(md5_with_count_missing_DF, self.conversion_path)
        ranked_DF.write.parquet(self.output_path)

    @staticmethod
    def load_rfi_md5_mapping(all_rfi_path, md5_rfi_mappings_path):
        all_rfi_DF = sparkSess.read.csv(all_rfi_path, sep="\t") \
                           .toDF(*["RFI", "Request ID"]) \
                           .select("RFI")
        md5_rfi_mappings_DF= sparkSess.read.csv(md5_rfi_mappings_path) \
                               .toDF(*["RFI", "MD5"])
        
        return all_rfi_DF.join(md5_rfi_mappings_DF, on="RFI", how="inner")
    
    
    def tu_transforming(self):
        tu_mapping = self.tu_mapping
        tu_transformed= self.tu
        
        cols = map(lambda x: x.lower(), tu_transformed.columns)
        
        tu_transformed = tu_transformed.toDF(*cols)

        tu_mapping = tu_mapping.filter(col('raw_var').isin(*tu_transformed.columns))

        binary_cols_df = tu_mapping.filter(col('values') == 'binary').select('raw_var', 'grouped_var').collect()
        binary_cols = [(r['raw_var'], r['grouped_var']) for r in binary_cols_df]

        cols_to_combine_df = tu_mapping.filter(col('values') != 'binary').collect()
        cols_to_combine = [(r['raw_var'], r['grouped_var'], r['values']) for r in cols_to_combine_df]

        tu_transformed = tu_transformed.select('email_address_md5', 
                                           *[when(col(col_map[0]).isNull(), 0).otherwise(1).name(col_map[1]) for col_map in binary_cols],
                                           *[when(col(col_map[0]) == 1, col_map[2]).otherwise(None).name(col_map[0]) for col_map in cols_to_combine])

        for new_var in set([col_map[1] for col_map in cols_to_combine]):
            raw_vars = set([col_map[0] for col_map in cols_to_combine if col_map[1] == new_var])
            tu_transformed = tu_transformed.withColumn(new_var, concat_ws('_', *raw_vars)).withColumn(new_var, when(col(new_var)!='', col(new_var)).otherwise('Missing'))

        raw_vars = set([col_map[0] for col_map in cols_to_combine+binary_cols])
        tu_transformed = tu_transformed.drop(*(raw_vars)).withColumnRenamed('email_address_md5', "EMAIL_ADDRESS_MD5")
        try:
            ### these columns create problems bcs of their syntax
            tu_transformed= tu_transformed.drop('tu_homehasappreciatedinlast12mos.', 'tu_homehasdepreciatedinlast12mos.')
        except:
            pass

        tu_mapping.unpersist()
        return tu_transformed
    
        
    def generate_md5_count_missing(self):
        """Full outer join between acxiom and TU; only on the basis of these two we"""
        
        #### does not work!!! for now I comment it out
#         tu_transformed= self.tu_transforming()


        joined_df = self.acxiom.join(self.tu, on= ['EMAIL_ADDRESS_MD5', 'PROFILE_ID'], how= 'full_outer') \
                        .withColumnRenamed("EMAIL_ADDRESS_MD5", "MD5")
        
        acxiom_tu_columns = list(set(joined_df.columns) - set("MD5"))
        calc_num_missing = lambda x: F.when(F.col(x).isNull(), 1).otherwise(0)
        
        return \
          joined_df.join(self.joined_records_DF, on="MD5", how= 'inner') \
                   .withColumn("COUNT_MISSING", sum([calc_num_missing(x) for x in acxiom_tu_columns])) \
                   .select(["RFI", "MD5", 'COUNT_MISSING'])
                  
    
    def filter_by_count_missing_rank(self, df, conversion_path):
        
        conversions = sparkSess.read.csv(conversion_path, header= False).withColumn('Conversion', F.lit(1))  \
                                         .withColumnRenamed('_c2', 'EMAIL_ADDRESS_MD5').drop('_c0', '_c1')
        
        df= df.withColumnRenamed("MD5", "EMAIL_ADDRESS_MD5")
        df= df.join(conversions, on='EMAIL_ADDRESS_MD5', how= 'left') \
                                                .fillna(0, subset=['Conversion'])
        
        
        
        df.createOrReplaceTempView("to_rank_official")

        ranked_final = sparkSess.sql(" SELECT RFI, COUNT_MISSING, EMAIL_ADDRESS_MD5, CONVERSION, \
                        rank() over (PARTITION BY RFI ORDER BY count_missing asc, EMAIL_ADDRESS_MD5) AS RANK \
                        FROM to_rank_official  ")
        
        
        ### I get for each RFI the maximum conversion, since if at least 1 household converted, it s ok
        propagating_convertion= ranked_final.groupby('RFI')      \
                                            .agg((F.max('Conversion'))   \
                                                .alias('Max_Conversion'))
        
        
        propagating_convertion.createOrReplaceTempView("propagating_convertion")
        
        ranked_final.createOrReplaceTempView("to_rank_official")
        propagating_convertion.createOrReplaceTempView("propagating_convertion")
        
        ranked_final = sparkSess.sql(" SELECT A.RFI, COUNT_MISSING, EMAIL_ADDRESS_MD5, MAX_CONVERSION, \
                                rank() over (PARTITION BY A.RFI ORDER BY count_missing asc, EMAIL_ADDRESS_MD5) AS RANK \
                                FROM to_rank_official  A   \
                                INNER JOIN propagating_convertion B \
                                ON A.RFI==B.RFI ")
        
        
        ranked_final.createOrReplaceTempView("ranked_table")


        final_mapping_rfi_md5 = sparkSess.sql(" SELECT RFI, EMAIL_ADDRESS_MD5, MAX_CONVERSION \
                                            FROM ranked_table   \
                                            WHERE RANK == 1 " )
        
        return final_mapping_rfi_md5

In [11]:
mapping_conv = Mapping_rfi_md5_conv(acxiom_path=acxiom_path,
                          tu_path=tu_path, 
                          all_rfi_path=all_rfi_path, 
                          md5_rfi_mappings_path=md5_rfi_mappings_path,
                          conversion_path= conversion_path ,         
                          output_path="hdfs:///md5_mapping_with_conv/")

In [12]:
mapping_conv.execute()

In [27]:
t1= mapping_conv.load_rfi_md5_mapping(all_rfi_path=all_rfi_path, 
                              md5_rfi_mappings_path=md5_rfi_mappings_path,)
t1.show()

+-------------------+--------------------+
|                RFI|                 MD5|
+-------------------+--------------------+
|1001558335910283967|30df85ec60751d05b...|
|1001558335910283967|bf3558b830afd37f4...|
|1001558335910283967|12842a3ce3e3ed472...|
|1001558335910283967|a96b6d100ba9de09e...|
|1001558335916624266|3d9d18dcc821e481d...|
|1001558335916624266|e4f068b9c0f4ef987...|
|1001558335916624266|abe024ad5858bbee6...|
|1001558335916624266|20348bee6be5afa00...|
|1001558335916624266|1e9b88cfda2c212ef...|
|1001558335916624266|a82a908be803a615f...|
|1001558335916624266|6c37e143991612903...|
|1001558335916624266|780153f39b9c3cc34...|
|1001558335931974134|d77d018812065735d...|
|1001558335931974134|5d679ce6c149f30f1...|
|1001558335931974134|5dcc11d2f012af408...|
|1001558335931974134|268a6eeb1ce696600...|
|1001558335931974134|ab3957c9784369158...|
|1001558335931974134|a46f9685deb710d91...|
|1001558335931974134|3ed348e7215b4cd25...|
|1001558335931974134|dc38adef06d964217...|
+----------

In [ ]:
# tu_treated = mapping_conv.tu_transforming()
# tu_treated.columns[:5]

In [8]:
t2= mapping_conv.generate_md5_count_missing()

+-------------------+--------------------+-------------+
|                RFI|                 MD5|COUNT_MISSING|
+-------------------+--------------------+-------------+
|7018250396118575588|00000186b2395bcff...|            0|
| 784541142021525012|0000ba03e35b97cb5...|            0|
|2339127428260532026|0000df561ad83c980...|            0|
|2339971853089377473|00012be322c81e5f1...|            0|
|5492586308497501055|0002158ce347ac308...|            0|
|5440883427593125031|000296e50bd162d60...|            0|
|7290706841403153070|0002c8ccb0d555d05...|          122|
| 980447726988504051|00030151cb6f0b79b...|            0|
| 970033144884409943|000345c8a6107c58e...|            0|
| 770185912496418723|000450d65a1c39faf...|          122|
| 958774143348854386|00049a8512d33679e...|            0|
| 968344302320862551|00057bd2472c15071...|          122|
|2252896107512215054|0005f8ad4e5d70747...|          122|
| 978758865116576838|00084f9d2f0546639...|          122|
|2339408904093238502|00088a2d83

In [19]:
t3= mapping_conv.filter_by_count_missing_rank(df= t2, conversion_path= conversion_path )

+-------------------+--------------------+--------------+
|                RFI|   EMAIL_ADDRESS_MD5|MAX_CONVERSION|
+-------------------+--------------------+--------------+
|1001558335910283967|12842a3ce3e3ed472...|             0|
|1001558335916624266|1e9b88cfda2c212ef...|             0|
|1001558335931974134|01e93470828bf352f...|             0|
|1001558335934013971|3898d46dd10d5684f...|             0|
|1001558335950743328|398dd74cd8c29871c...|             0|
|1001558336003271199|26f0d2f32b464e72d...|             0|
|1001558336009756815|5b1131cddaa367ab0...|             0|
|1001558336026378378|495e3aad9ee41ccc5...|             0|
|1001558336045992930|1ebfcf9cc2643722e...|             0|
|1001558336049231297|812e4ee9277cea75a...|             0|
|1001558336114622376|1243923aa40ec51e4...|             0|
|1001558336133602821|6b8e362016c03e2d7...|             0|
|1001558336171385006|753a2faecddc254d1...|             0|
|  10080132718011701|8b6f1e4119e35a680...|             0|
|1008690668964